This notebook process staging dataset to targeted fact table.

In [2]:
import pandas as pd
import numpy as np
import yaml
from glob import glob
import os
from pathlib import Path
from datetime import date, timedelta, datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pathlib import Path
import re
import hashlib

### Import staging dataset

In [10]:
df_ridership = pd.read_csv("staging_ridership_output/ridership_with_loc_2026-01-15.csv", dtype={"stop_id": "string",
                                                                                                  "gtfs_stop_id": "string"})
df_ridership.head()

/tmp/ipykernel_284/2947545288.py:1: DtypeWarning: Columns (5,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ridership = pd.read_csv("staging_ridership_output/ridership_with_loc_2026-01-15.csv", dtype={"stop_id": "string",


,Unnamed: 0,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,stop_lat,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date,gtfs_stop_id
0,0,CEC173BF54FECCBD,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,45.0,Weekday,daily,2024-07-01,2024-07-01,7602
1,1,1BF770A6DC9B06BC,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,59.0,Weekday,daily,2024-07-02,2024-07-02,7602
2,2,19C42A2D3DD5337A,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,74.0,Weekday,daily,2024-07-03,2024-07-03,7602
3,3,40911F039E21320D,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,177.0,Weekday,daily,2024-07-04,2024-07-04,7602
4,4,BA5A97CE4B046876,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,110.0,Weekday,daily,2024-07-05,2024-07-05,7602


In [28]:
# take stop id provided by agency by default, if no stop id provided, use gtfs stop id
df_ridership["stop_id_comb"] = df_ridership["stop_id"].combine_first(df_ridership["gtfs_stop_id"])

In [29]:
df_ridership[df_ridership["stop_id_comb"].isna()]["organization_name"].unique()

array(['OmniTrans'], dtype=object)

**OmniTrans has stop names that are not matched in any stop name in GTFS.**

In [12]:
df_ridership[df_ridership["stop_id_comb"].isna()]

,Unnamed: 0,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,...,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date,gtfs_stop_id,stop_id_comb
1389443,1389443,D6F2B598A671C4EB,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,2ND @ F ST,...,NaN,2.572603,0.619178,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1389444,1389444,8C772D5F9586B138,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,2ND @ G ST,...,NaN,1.410959,7.701370,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1389466,1389466,E54CF08B65329762,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,EAST VALLEY,...,NaN,4.180822,3.473973,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1389467,1389467,9A1C62A9E262B5E6,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,GILBERT @ KENWOOD,...,NaN,0.709589,1.536986,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1389479,1389479,344DDD5B8D0F9084,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,MT VERNON @ JOHNSON,...,NaN,5.550685,18.405479,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394265,1394265,7911592B1D59A063,40F5F343F0BD5850,OmniTrans,OmniTrans,215,NaN,NaN,<NA>,EAST VALLEY,...,NaN,0.520548,0.416438,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>
1394267,1394267,2D3AB0813C064DDC,40F5F343F0BD5850,OmniTrans,OmniTrans,215,NaN,NaN,<NA>,MT VERNON @ CENTREPOINTE,...,NaN,16.873973,17.495890,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>
1394270,1394270,4434F9B74EB665B2,40F5F343F0BD5850,OmniTrans,OmniTrans,290,NaN,NaN,<NA>,ARROWHEAD MEDICAL CENTER,...,NaN,0.958904,0.843836,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>
1394271,1394271,2105316659A4E028,40F5F343F0BD5850,OmniTrans,OmniTrans,290,NaN,NaN,<NA>,EAST VALLEY,...,NaN,0.063014,0.057534,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>


### Handle OmniTrans Stop Name Format Inconsistency

In [13]:
omni_mask = df_ridership["organization_name"] == "OmniTrans"
t_df_omni = df_ridership.loc[omni_mask, ["stop_id_comb", "stop_name"]].copy()
t_df_omni["_orig_index"] = t_df_omni.index

In [14]:
street_types = ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "drive", "dr",
                "lane", "ln", "court", "ct", "place", "pl", "highway", "hwy", "way", "parkway", "pkwy"]

street_type_regex = r"\b(?:" + "|".join(street_types) + r")\b"

In [15]:
t_df_omni["stop_name_norm"] = t_df_omni["stop_name"].astype("string").str.lower().str.replace(r"\s+", " ", regex=True).str.strip()
# detect presence of any street type
t_df_omni["_has_street_type"] = t_df_omni["stop_name_norm"].str.contains(street_type_regex, regex=True, na=False)

t_df_omni["_stop_name_length"] = t_df_omni["stop_name_norm"].str.len()
# rank stop name variants within each stop id
canonical = t_df_omni.sort_values(by=["_has_street_type", "_stop_name_length"],
                                 ascending=[False, False]) \
                    .groupby("stop_id_comb", as_index=False).first()[["stop_id_comb", "stop_name"]].rename(columns={"stop_name": "canonical_stop_name"})
t_df_omni = t_df_omni.merge(canonical, on="stop_id_comb", how="left")
canonical.head()

,stop_id_comb,canonical_stop_name
0,1,ARROWHEAD REG MED CTR
1,10,Pacific High School
2,100,Foothill @ Riverside Ave
3,104,5th @ Medical Center
4,105,Merrill @ Riverside


In [16]:
df_ridership.loc[t_df_omni["_orig_index"], "stop_name"] = t_df_omni["canonical_stop_name"].values

### Agg by Stop ID and Stop Name

In [18]:
df_agg = df_ridership.groupby(["organization_name", "service_name", "stop_id_comb", "stop_name",
                               "stop_lat", "stop_lon", "day_type", "agg_basis", "start_date", "end_date"], as_index=False) \
                    .agg(avg_ridership = ("avg_ridership", lambda x: x.sum(min_count=1)),
                         avg_alightings = ("avg_alightings", lambda x: x.sum(min_count=1)),
                         avg_boardings = ("avg_boardings", lambda x: x.sum(min_count=1))).reset_index()

# drop index col
df_agg.drop(columns="index", inplace=True)

df_agg.head()

,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
0,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,Weekday,daily,2024-07-01,2024-07-01,6.0,NaN,NaN
1,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,Weekday,daily,2024-07-02,2024-07-02,25.0,NaN,NaN
2,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,Weekday,daily,2024-07-03,2024-07-03,16.0,NaN,NaN
3,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,Weekday,daily,2024-07-04,2024-07-04,30.0,NaN,NaN
4,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,Weekday,daily,2024-07-05,2024-07-05,30.0,NaN,NaN


In [19]:
t_df = df_agg.groupby(["organization_name", "service_name", "stop_id_comb", "day_type", "start_date", "end_date"]).size().reset_index(name="row_count").sort_values(by="row_count", ascending=False)
t_df.head(10)

,organization_name,service_name,stop_id_comb,day_type,start_date,end_date,row_count
552772,City of Santa Monica,Big Blue Bus,585,WEEKDAY,2024-12-01,2025-03-31,4
552767,City of Santa Monica,Big Blue Bus,585,SUNDAY,2024-08-01,2024-11-30,4
550311,City of Santa Monica,Big Blue Bus,243,WEEKDAY,2025-08-01,2025-11-30,4
550310,City of Santa Monica,Big Blue Bus,243,WEEKDAY,2025-04-01,2025-07-31,4
550309,City of Santa Monica,Big Blue Bus,243,WEEKDAY,2024-12-01,2025-03-31,4
552774,City of Santa Monica,Big Blue Bus,585,WEEKDAY,2025-08-01,2025-11-30,4
552773,City of Santa Monica,Big Blue Bus,585,WEEKDAY,2025-04-01,2025-07-31,4
552771,City of Santa Monica,Big Blue Bus,585,WEEKDAY,2024-08-01,2024-11-30,4
552770,City of Santa Monica,Big Blue Bus,585,SUNDAY,2025-08-01,2025-11-30,4
552769,City of Santa Monica,Big Blue Bus,585,SUNDAY,2025-04-01,2025-07-31,4


In [20]:
t_df[t_df["row_count"] > 1]["organization_name"].unique()

array(['City of Santa Monica', 'Santa Cruz Metropolitan Transit District'],
      dtype=object)

In [21]:
t_df[t_df["row_count"] > 1].groupby(by=["organization_name", "service_name", "day_type", "start_date", "end_date"])["stop_id_comb"].nunique().reset_index()

,organization_name,service_name,day_type,start_date,end_date,stop_id_comb
0,City of Santa Monica,Big Blue Bus,SATURDAY,2024-08-01,2024-11-30,236
1,City of Santa Monica,Big Blue Bus,SATURDAY,2024-12-01,2025-03-31,237
2,City of Santa Monica,Big Blue Bus,SATURDAY,2025-04-01,2025-07-31,222
3,City of Santa Monica,Big Blue Bus,SATURDAY,2025-08-01,2025-11-30,255
4,City of Santa Monica,Big Blue Bus,SUNDAY,2024-08-01,2024-11-30,236
5,City of Santa Monica,Big Blue Bus,SUNDAY,2024-12-01,2025-03-31,237
6,City of Santa Monica,Big Blue Bus,SUNDAY,2025-04-01,2025-07-31,222
7,City of Santa Monica,Big Blue Bus,SUNDAY,2025-08-01,2025-11-30,255
8,City of Santa Monica,Big Blue Bus,WEEKDAY,2024-08-01,2024-11-30,325
9,City of Santa Monica,Big Blue Bus,WEEKDAY,2024-12-01,2025-03-31,326


#### Big Blue Bus

In [24]:
df_big_blue_bus = df_agg[df_agg["organization_name"] == "City of Santa Monica"]

In [27]:
df_big_blue_bus[(df_big_blue_bus["stop_id_comb"] == "585") & (df_big_blue_bus["start_date"] == "2024-08-01")]

,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
554782,City of Santa Monica,Big Blue Bus,585,CENTINELA NB/WASHINGTON BLVD FS,33.997316,-118.428587,SATURDAY,service period,2024-08-01,2024-11-30,NaN,8.440476,28.298809
554786,City of Santa Monica,Big Blue Bus,585,CENTINELA NB/WASHINGTON BLVD FS,33.997316,-118.428587,SUNDAY,service period,2024-08-01,2024-11-30,NaN,6.244372,21.710858
554790,City of Santa Monica,Big Blue Bus,585,CENTINELA NB/WASHINGTON BLVD FS,33.997316,-118.428587,WEEKDAY,service period,2024-08-01,2024-11-30,NaN,15.787934,41.066271
554794,City of Santa Monica,Big Blue Bus,585,CENTINELA NB/WASHINGTON PLACE MB,33.998714,-118.429655,SATURDAY,service period,2024-08-01,2024-11-30,NaN,6.064286,7.440476
554798,City of Santa Monica,Big Blue Bus,585,CENTINELA NB/WASHINGTON PLACE MB,33.998714,-118.429655,SUNDAY,service period,2024-08-01,2024-11-30,NaN,5.443470,10.201479
554802,City of Santa Monica,Big Blue Bus,585,CENTINELA NB/WASHINGTON PLACE MB,33.998714,-118.429655,WEEKDAY,service period,2024-08-01,2024-11-30,NaN,10.358636,20.175733
554806,City of Santa Monica,Big Blue Bus,585,CENTINELA SB/WASHINGTON NS,33.997269,-118.428828,SATURDAY,service period,2024-08-01,2024-11-30,NaN,26.704761,12.246428
554810,City of Santa Monica,Big Blue Bus,585,CENTINELA SB/WASHINGTON NS,33.997269,-118.428828,SUNDAY,service period,2024-08-01,2024-11-30,NaN,18.693831,10.776046
554814,City of Santa Monica,Big Blue Bus,585,CENTINELA SB/WASHINGTON NS,33.997269,-118.428828,WEEKDAY,service period,2024-08-01,2024-11-30,NaN,52.545791,26.167921
554818,City of Santa Monica,Big Blue Bus,585,CENTINELA SB/WASHINGTON PLACE NS,33.999714,-118.430702,SATURDAY,service period,2024-08-01,2024-11-30,NaN,20.916666,8.846428


#### Santa Cruz Metro

In [22]:
t_df[(t_df["service_name"] == "Santa Cruz METRO") & (t_df["row_count"] > 1)]

,organization_name,service_name,stop_id_comb,day_type,start_date,end_date,row_count
1186979,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,2594,all,2024-07-01,2024-09-11,2
1185881,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,all,2024-07-01,2024-09-11,2
1185882,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,all,2024-09-12,2024-12-18,2
1185667,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1666,all,2024-12-19,2025-03-12,2


In [20]:
df_smtd = df_agg[df_agg["organization_name"] == "San Mateo County Transit District"]
df_smtd[(df_smtd["stop_id_comb"] == "363049") & (df_smtd["start_date"] == "2025-08-07")]

,index,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
1180042,1180042,San Mateo County Transit District,SamTrans,363049,Pulgas Ave & E Bayshore Rd,37.455791,-122.129951,weekday,daily,2025-08-07,2025-08-07,NaN,1.0,16.0


**Note: The duplicates in SamTrans is the stop id that is not in GTFS - and in ridership data, a stop id can have more than one pair of lat/lon. Looks like it is vehicle position. So we use GTFS lat/lon mapped in using stop id for the aggregation.**

In [11]:
df_sc[df_sc["stop_id_comb"] == "1796"]

,index,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
1189548,1189548,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,Soquel Ave + Pacheco Ave,36.981586,-122.003937,all,custom period,2024-07-01,2024-09-11,NaN,83.0,63.0
1189549,1189549,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,Soquel Ave + Pacheco Ave,36.981586,-122.003937,all,custom period,2024-09-12,2024-12-18,NaN,96.0,67.0
1189550,1189550,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,Soquel Ave + Pacheco Ave,36.981586,-122.003937,all,custom period,2024-12-19,2025-03-12,NaN,2741.0,2273.0
1189551,1189551,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,Soquel Ave + Pacheco Ave,36.981586,-122.003937,all,custom period,2025-03-13,2025-06-30,NaN,6088.0,4315.0
1189552,1189552,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,Soquel Ave + San Juan Ave,36.981586,-122.003937,all,custom period,2024-07-01,2024-09-11,NaN,1766.0,1412.0
1189553,1189553,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,1796,Soquel Ave + San Juan Ave,36.981586,-122.003937,all,custom period,2024-09-12,2024-12-18,NaN,3215.0,2678.0


In [28]:
df_omnitrans = df_agg[df_agg["organization_name"] == "OmniTrans"]
df_omnitrans[(df_omnitrans["stop_id_comb"] == '6021') & (df_omnitrans["start_date"] == '2025-07-01')]

,index,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
1097512,1097512,OmniTrans,OmniTrans,6021,BASELINE @ SIERRA WY,34.121441,-117.284899,all,fiscal year,2025-07-01,2026-06-30,NaN,3.328767,2.624658
1097515,1097515,OmniTrans,OmniTrans,6021,Baseline @ Sierra,34.121441,-117.284899,all,fiscal year,2025-07-01,2026-06-30,NaN,0.586301,0.328767
1097518,1097518,OmniTrans,OmniTrans,6021,Baseline @ Sierra Way,34.121441,-117.284899,all,fiscal year,2025-07-01,2026-06-30,NaN,2.852055,2.909589


In [29]:
df_omnitrans[(df_omnitrans["stop_id_comb"] == '5488') & (df_omnitrans["start_date"] == '2025-07-01')]

,index,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
1096738,1096738,OmniTrans,OmniTrans,5488,E @ SOUTH MALL,34.083248,-117.294109,all,fiscal year,2025-07-01,2026-06-30,NaN,0.227397,0.131507
1096741,1096741,OmniTrans,OmniTrans,5488,E ST @ SOUTH MALL,34.083248,-117.294109,all,fiscal year,2025-07-01,2026-06-30,NaN,0.079452,0.408219


In [30]:
df_omnitrans[(df_omnitrans["stop_id_comb"] == '7288') & (df_omnitrans["start_date"] == '2025-07-01')]

,index,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
1098958,1098958,OmniTrans,OmniTrans,7288,ARROW @ ARCHIBALD,34.099274,-117.593291,all,fiscal year,2025-07-01,2026-06-30,NaN,0.731507,0.797260
1098961,1098961,OmniTrans,OmniTrans,7288,Arrow Route @ Archibald,34.099274,-117.593291,all,fiscal year,2025-07-01,2026-06-30,NaN,0.430137,0.556164


In [38]:
t_df_omnitrans = df_omnitrans.groupby("stop_id_comb")["stop_name"].nunique().reset_index(name="num_stop_name")
id_list_omnitrans = t_df_omnitrans[t_df_omnitrans["num_stop_name"] > 1]["stop_id_comb"].unique().tolist()

In [39]:
df_omnitrans[(df_omnitrans["stop_id_comb"].isin(id_list_omnitrans))]

,index,organization_name,service_name,stop_id_comb,stop_name,stop_lat,stop_lon,day_type,agg_basis,start_date,end_date,avg_ridership,avg_alightings,avg_boardings
1095974,1095974,OmniTrans,OmniTrans,100,FOOTHILL @ RIVERSIDE,34.106804,-117.370483,all,fiscal year,2023-07-01,2024-06-30,NaN,22.986301,21.019178
1095975,1095975,OmniTrans,OmniTrans,100,FOOTHILL @ RIVERSIDE,34.106804,-117.370483,all,fiscal year,2024-07-01,2025-06-30,NaN,31.205479,27.917808
1095976,1095976,OmniTrans,OmniTrans,100,FOOTHILL @ RIVERSIDE,34.106804,-117.370483,all,fiscal year,2025-07-01,2026-06-30,NaN,6.701370,5.873973
1095977,1095977,OmniTrans,OmniTrans,100,Foothill @ Riverside Ave,34.106804,-117.370483,all,fiscal year,2023-07-01,2024-06-30,NaN,20.860274,22.408219
1095978,1095978,OmniTrans,OmniTrans,100,Foothill @ Riverside Ave,34.106804,-117.370483,all,fiscal year,2024-07-01,2025-06-30,NaN,28.142466,32.175342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099687,1099687,OmniTrans,OmniTrans,8621,San Bernardino @ Orchard,34.077504,-117.205501,all,fiscal year,2024-07-01,2025-06-30,NaN,2.158904,1.610959
1099688,1099688,OmniTrans,OmniTrans,8621,San Bernardino @ Orchard,34.077504,-117.205501,all,fiscal year,2025-07-01,2026-06-30,NaN,0.378082,0.334247
1099689,1099689,OmniTrans,OmniTrans,8621,San Bernardino @ Orchard Way,34.077504,-117.205501,all,fiscal year,2023-07-01,2024-06-30,NaN,8.887671,5.564384
1099690,1099690,OmniTrans,OmniTrans,8621,San Bernardino @ Orchard Way,34.077504,-117.205501,all,fiscal year,2024-07-01,2025-06-30,NaN,9.857534,6.465753


#### Add key

In [30]:
def generate_key(*args, len=16, sep="|"):
    """Generate short reproducible hash ID based on input string"""
    key = sep.join(str(arg) for arg in args if arg is not None).encode("utf-8")
    return hashlib.sha1(key).hexdigest()[:len].upper()

In [31]:
df_agg = df_agg.rename(columns={"stop_id_comb": "stop_id"})

In [32]:
df_agg["key"] = df_agg.apply(lambda row: generate_key(row["organization_name"],
                                                           row["service_name"],
                                                           row["stop_id"],
                                                           row["stop_name"],
                                                           row["day_type"],
                                                           row["start_date"]
                                                          ), axis=1, result_type="expand")

In [33]:
# check duplicates
t_df_agg = df_agg.groupby("key").size().reset_index(name="count")
t_df_agg[t_df_agg["count"] > 1]

,key,count


In [36]:
df_agg = df_agg[["key", "organization_name", "service_name", "stop_id", "stop_name", "stop_lat", "stop_lon", 
                 "avg_alightings", "avg_boardings", "avg_ridership", "start_date", "end_date", "agg_basis", "day_type"]]

#### Export

In [37]:
today = date.today().strftime("%Y-%m-%d")
df_agg.to_csv(f"fact_ridership_output/fct_ridership_{today}.csv", index=False) # save a copy

In [38]:
df_agg.head()

,key,organization_name,service_name,stop_id,stop_name,stop_lat,stop_lon,avg_alightings,avg_boardings,avg_ridership,start_date,end_date,agg_basis,day_type
0,7E7D4E360CBEBB0C,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,NaN,NaN,6.0,2024-07-01,2024-07-01,daily,Weekday
1,9E01C347A651319A,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,NaN,NaN,25.0,2024-07-02,2024-07-02,daily,Weekday
2,6C571A2A80296937,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,NaN,NaN,16.0,2024-07-03,2024-07-03,daily,Weekday
3,31113C3371BBC346,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,NaN,NaN,30.0,2024-07-04,2024-07-04,daily,Weekday
4,2CDD67FCFDBE6CFA,City and County of San Francisco,Golden Gate Park Shuttle,7601,Transverse,37.770534,-122.479657,NaN,NaN,30.0,2024-07-05,2024-07-05,daily,Weekday
